QUESTION 1: Total BPD Budget vs. Previous Years

FY15 Adopted: $319700081
FY15 Approp: $319700081
FY15 Exp: $337310895

FY16 Adopted: $323509388
FY16 Approp: $332544971
FY16 Exp: $348887844

FY17 Adopted: $356341193
FY17 Approp: $364321048
FY17 Exp: $364594820

FY18 Adopted: $373814105
FY18 Approp: $385562872
FY18 Exp: $399924493

FY19 Adopted: $400425675
FY19 Approp: $400435574
FY19 Exp: $416762373

FY20 Adopted: $414237376
FY20 Approp: $414306876
FY20 Exp: $425553210

FY21 Adopted: $404182025
FY21 Approp: $404182025
FY21 Exp: $422917499

FY22 Adopted: $399871218
FY22 Approp: $400451303
 
FY23 Adopted: $395094796

In [5]:
import pandas as pd
import matplotlib.pyplot as plt

file_adopted_operating_budget_fy23 = pd.read_csv('./Data/operating-budget.csv')

budget_df_uncut = pd.DataFrame(file_adopted_operating_budget_fy23)

budget_df_withNA = budget_df_uncut[['Cabinet', 'Dept', 'Program   ', 'Expense Category',
       'FY20 Actual Expense', 'FY21 Actual Expense', 'FY22 Appropriation',
       'FY23 Adopted']]

budget_df = budget_df_withNA.dropna()

# Replace '#Missing' values with '0' so we can type cast the columns to numbers
budget_df = budget_df.replace('#Missing', '0')

# Convert datatype to strings 
budget_df['FY20 Actual Expense'] = budget_df['FY20 Actual Expense'].apply(lambda x: float(x.replace(",","").replace("$","")) if x != 'TOTAL_GROSS' and str(type(x)) == '<class \'str\'>' else x)
budget_df['FY21 Actual Expense'] = budget_df['FY21 Actual Expense'].apply(lambda x: float(x.replace(",","").replace("$","")) if x != 'TOTAL_GROSS' and str(type(x)) == '<class \'str\'>' else x)
budget_df['FY22 Appropriation'] = budget_df['FY22 Appropriation'].apply(lambda x: float(x.replace(",","").replace("$","")) if x != 'TOTAL_GROSS' and str(type(x)) == '<class \'str\'>' else x)
budget_df['FY23 Adopted'] = budget_df['FY23 Adopted'].apply(lambda x: float(x.replace(",","").replace("$","")) if x != 'TOTAL_GROSS' and str(type(x)) == '<class \'str\'>' else x)

# Get only data from Police Department
bpd_budget_df = budget_df.loc[budget_df['Dept'] == "Police Department"]

# Compute sums and add to list
bpd_budget = []
# bpd_budget.append(bpd_budget_df['FY20 Actual Expense'].sum())
# bpd_budget.append(bpd_budget_df['FY21 Actual Expense'].sum())
# bpd_budget.append(bpd_budget_df['FY22 Appropriation'].sum())  # Appropriated budget = means ?? (GUESS: Proposed budget (Needs approval until adopted!))
# bpd_budget.append(bpd_budget_df['FY23 Adopted'].sum())  # Adopted budget = 1 year spending fianncial plan
bpd_budget = [319700081, 323509388, 356341193, 373814105, 400425675, 414237376, 404182025, 399871218, 395094796]

fig1_dict = {
    'year': ['FY15', 'FY16', 'FY17', 'FY18', 'FY19', 'FY20', 'FY21', 'FY22', 'FY23'],
    'budget': bpd_budget
}

pd.options.plotting.backend = "plotly"
df1 = pd.DataFrame(fig1_dict)
print(df1)
# using Plotly Express via the Pandas backend
fig1 = df1.plot.bar(x='year', y='budget', title='BPD Budget Each Year', color_discrete_sequence=['#EF404E'])
fig1.update_layout(plot_bgcolor='rgba(0,0,0,0)')
fig1.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='LightGray')
fig1.show()

   year     budget
0  FY15  319700081
1  FY16  323509388
2  FY17  356341193
3  FY18  373814105
4  FY19  400425675
5  FY20  414237376
6  FY21  404182025
7  FY22  399871218
8  FY23  395094796


Note: FY 20 & 21 are Expenses, FY 22 and 23 are Budgets 
Police dept budget slightly decreased in recent years.

Improvement:
- We could change this to only budgets


QUESTION 2: BPD Budget vs. other departments budgets + change over time

Issues:
- Which departments do we include (excluding BPD) ?
- I look at Top 4 departments with highest budgets each year + BPD

In [6]:
import numpy as np
import plotly_express as px

num_depts = 5
year = ['FY20 Expense'] * num_depts + ['FY21 Expense'] * num_depts + ['FY22 Appropriated Budget'] * num_depts + ['FY23 Adopted Budget'] * num_depts

budget = []
dept = []

# Sum all budgets
summed_budget_df = budget_df.groupby(by=['Dept'], sort=False).sum()

# Will set the plot values based on all departments
def getTopOverallDepts():
    # Sort departments by budget sums
    fy20_sorted_df = summed_budget_df.sort_values(by=['FY20 Actual Expense'], ascending=False)
    fy21_sorted_df = summed_budget_df.sort_values(by=['FY21 Actual Expense'], ascending=False)
    fy22_sorted_df = summed_budget_df.sort_values(by=['FY22 Appropriation'], ascending=False)
    fy23_sorted_df = summed_budget_df.sort_values(by=['FY23 Adopted'], ascending=False)

    # Get top departments
    top_fy20_df = fy20_sorted_df.head(num_depts)
    top_fy21_df = fy21_sorted_df.head(num_depts)    
    top_fy22_df = fy22_sorted_df.head(num_depts)    
    top_fy23_df = fy23_sorted_df.head(num_depts)    

    for index, row in top_fy20_df.iterrows():
        dept.append(index)
        budget.append(row['FY20 Actual Expense'])

    for index, row in top_fy21_df.iterrows():
        dept.append(index)
        budget.append(row['FY21 Actual Expense'])

    for index, row in top_fy22_df.iterrows():
        dept.append(index)
        budget.append(row['FY22 Appropriation'])

    for index, row in top_fy23_df.iterrows():
        dept.append(index)
        budget.append(row['FY23 Adopted'])

getTopOverallDepts()

fig2_dict = {
    'year': year,
    'budget': budget,
    'dept': dept
}

df2 = pd.DataFrame(fig2_dict)
color_discrete_sequence=['#6c8bff','#6cffe0','#ffe06c','#ff6c8b', "#e586ff", "#ff976c"]
fig2 = px.bar(df2, x='year', y='budget', color='dept', title='Top 5 Department Budgets Each Year', barmode="group", color_discrete_sequence=color_discrete_sequence)
fig2.update_layout(plot_bgcolor='rgba(0,0,0,0)')
fig2.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='LightGray')
fig2.show()



/var/folders/15/ngd639_x4tn_n4xh55wvc5lr0000gn/T/ipykernel_76511/1166137425.py:11: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



Analysis: 
BPD remained the department with the second highest budget each year, with small decrements over the years. 
BPS, Pensions, and Charter School Tuition all had budget increases over the years.

NOTE: Might be more useful to show percentage of total city budget that BPD takes up per year + change over time

QUESTION 3: Number of police officers per 10,000 residents

Presentation: 
- Look at FBI uniform crime reporting average of police officers per 10,000 residents & compare it to boston
- FBI data sets here: https://crime-data-explorer.fr.cloud.gov/#
- Bar chart where each x-axis represent year and each year there will be 2 bars, 1 for average in nation, 1 for value in boston  
OR
- Scatter plot, each point is a city
- X axis is population
- Y axis is officers / 10,000 residents

NOTE: Some cities with low populations have 0 officers! (How do we handle this if at all?) 

In [7]:
import plotly.figure_factory as ff

fbi_law_enforcement_employee_data = pd.read_csv('./Data/FBI_law_enforcement_employee_data.csv')
employee_nums_1960_2021 = pd.DataFrame(fbi_law_enforcement_employee_data)
employee_nums_1960_2021.dropna()

# Get only city data
employee_nums_1960_2021 = employee_nums_1960_2021.loc[employee_nums_1960_2021['agency_type_name'] == 'City']

# Get only data from 2016-2021
employee_nums_2015_2021 = employee_nums_1960_2021.loc[employee_nums_1960_2021['data_year'].isin([2015, 2016, 2017, 2018, 2019, 2020, 2021])]

boston_vals = []
average_vals = []
median_vals = []
city_vals_each_year = [] # List of lists where the first list is the values of all cities in 2015
boston_percentiles = []

# Get Boston df
boston_df = employee_nums_2015_2021.loc[employee_nums_2015_2021["pub_agency_name"] == "Boston"]
boston_df = boston_df.loc[employee_nums_2015_2021["state_abbr"] == "MA"]
# boston_df = boston_df.sort_values(by='data_year', ascending=True)

for i in range(2015,2022):

    # Get Boston values
    row = boston_df.loc[boston_df["data_year"] == i]
    row = row.squeeze()
    pop = row["population"]
    officer_ct = row["officer_ct"]
    # pop * factor = 10,000 so factor = 10000/pop
    factor = 10000/pop
    bos_offc_per_10k = officer_ct * factor
    boston_vals.append(bos_offc_per_10k)

    cities_measured = 0

    # Get average values
    i_year_rows = employee_nums_2015_2021.loc[employee_nums_2015_2021["data_year"] == i]
    all_cities_vals = []
    for index, row in i_year_rows.iterrows():
        pop = row["population"]
        if (pop == 0):
            break
        officer_ct = row["officer_ct"] 
        factor = 10000/pop
        offc_per_10k = officer_ct * factor
        # if (officer_ct == 0):
        #     print(str(row["data_year"]) + row['pub_agency_name'])
        # Ignore nan values
        if np.isnan(offc_per_10k):
            break
        cities_measured += 1
        all_cities_vals.append(offc_per_10k)

    # Compute statistices
    sum = np.sum(all_cities_vals)
    avg = sum/len(all_cities_vals)
    median = np.median(all_cities_vals)

    city_vals_each_year.append(all_cities_vals)
    median_vals.append(median)
    average_vals.append(avg)

    cities_w_less_offc_per_capita = np.where(all_cities_vals < bos_offc_per_10k)[0]

    percentile = round(((len(cities_w_less_offc_per_capita)/cities_measured) * 100), 2)

    print("Total cities measured:" + str(cities_measured))
    print("Total cities with less officers/capita than Boston: " + str(len(cities_w_less_offc_per_capita)))
    print("Boston has more officers/capita than: " + str(percentile) + " percent of other city PDs")
    boston_percentiles.append(percentile)

print(boston_vals)
print(average_vals)
print(median_vals)

# Create plots
year = ["2015"] * 2 + ["2016"] * 2 + ["2017"] * 2 + ["2018"] * 2 + ["2019"] * 2 + ["2020"] * 2 + ["2021"] * 2
values = []
labels = []

for i in range(int(len(year)/2)):
    values.append(median_vals[i])
    values.append(boston_vals[i])
    labels.append("Median")
    labels.append("Boston")

# BAR CHART
fig3_dict = {
    'Year': year,
    'Officers per 10k Residents': values,
    'Key': labels
}

colors = ['#231F20','#EF404E']
df3 = pd.DataFrame(fig3_dict)
fig3 = px.bar(df3, x='Year', y='Officers per 10k Residents', color='Key', title='Officers Per 10,000 Residents (Median of other Departments vs Boston)', barmode="group", color_discrete_map={
        'Boston': '#EF404E',
        'Median': '#231F20'
    })
fig3.update_layout(plot_bgcolor='rgba(0,0,0,0)')
fig3.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='LightGray')
fig3.show()


# DISTRIBUTION GRAPHS
hist_data = [city_vals_each_year[-1]]
group_labels = ['Density Curve'] 
bos = boston_vals[-1] # Get Boston's 2021 value


colors = ['#0055AA']
fig4 = ff.create_distplot(hist_data, group_labels, show_rug=False, colors=colors)
fig4.update_layout(title="2021 Distribution of Officers Per 10,000 Residents", xaxis_title="Officers per 10,000 residents", yaxis_title="Frequency")
fig4.add_vline(x=bos, line_width=2, line_dash="dash", line_color="red", annotation_text="Boston officers per pop > " + str(boston_percentiles[-1]) + "% of cities")
fig4.update_layout(plot_bgcolor='rgba(0,0,0,0)')
fig4.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='LightGray')
fig4.show()



Total cities measured:2818
Total cities with less officers/capita than Boston: 2424
Boston has more officers/capita than: 86.02 percent of other city PDs
Total cities measured:2701
Total cities with less officers/capita than Boston: 2240
Boston has more officers/capita than: 82.93 percent of other city PDs
Total cities measured:2851
Total cities with less officers/capita than Boston: 2387
Boston has more officers/capita than: 83.73 percent of other city PDs
Total cities measured:619
Total cities with less officers/capita than Boston: 424
Boston has more officers/capita than: 68.5 percent of other city PDs
Total cities measured:555
Total cities with less officers/capita than Boston: 385
Boston has more officers/capita than: 69.37 percent of other city PDs
Total cities measured:419
Total cities with less officers/capita than Boston: 249
Boston has more officers/capita than: 59.43 percent of other city PDs
Total cities measured:346
Total cities with less officers/capita than Boston: 209
B

FIGURE 3 Analysis:
- Boston constantly ranks higher than the median in terms of Officers per 10k residents
- Boston's officers/10k pop, has decreased over the years, while the median has increased

    ISSUES:
    - UCR (Uniform Crime Reports) are not mandatory! This explains the variation in the # of cities which submitted UCR data each year
    - Interestingly, the # of cities which submitted UCR datas has been steadily decreasing
    - Upwards trend of median could potentially be explained by this
    - POSSIBLE EXPLANATION: Larger cities (which typically have larger police forces) may continue to report UCR while smaller cities may have opted out

FIGURE 4 Analysis:
- Different way of displaying the "Officers / 10k residents" statistic
- We are able to see where Boston lies compared to other cities 
- I also display a percentage

    ISSUES:
    - I only plot values for 1 year (2021)
    - Can fix this by using: Box plot OR Ridgeline plot

Question 4: 
BPD proposed vs adopted budget

Proposed Budget --> Adopted Budget --> Appropriated Budget --> Actual Expenditure

FY15 Adopted: $319700081
FY15 Approp: $319700081
FY15 Exp: $337310895

FY16 Adopted: $323509388
FY16 Approp: $332544971
FY16 Exp: $348887844

FY17 Adopted: $356341193
FY17 Approp: $364321048
FY17 Exp: $364594820

FY18 Adopted: $373814105
FY18 Approp: $385562872
FY18 Exp: $399924493

FY19 Adopted: $400425675
FY19 Approp: $400435574
FY19 Exp: $416762373

FY20 Adopted: $414237376
FY20 Approp: $414306876
FY20 Exp: $425553210

FY21 Adopted: $404182025
FY21 Approp: $404182025
FY21 Exp: $422917499

FY22 Adopted: $399871218
FY22 Approp: $400451303
 
FY23 Adopted: $395094796

Issue: We dont have a lot of data for proposed budget

Question: What do we learn by comparing proposed & adopted budgets? Wouldn't it be more useful to compare adopted budget & actual expenditures, or just measure all types of budgets + expenditure?

In [8]:
# BAR CHART
from plotly.graph_objs import *

year = ['FY15'] * 3 + ['FY16'] * 3 + ['FY17'] * 3 + ['FY18'] * 3 + ['FY19'] * 3 + ['FY20'] * 3 + ['FY21'] * 3 + ['FY22'] * 2 + ['FY23']
labels = ['Adopted', 'Appropriated', 'Expense'] * 7 + ['Adopted', 'Appropriated', 'Adopted']
values = [319700081, 319700081, 337310895, 323509388, 332544971, 348887844, 356341193, 364321048, 364594820, 373814105, 385562872, 399924493, 400425675, 400435574, 416762373, 414237376, 414306876, 425553210, 404182025, 404182025, 422917499, 399871218, 400451303, 395094796]

layout = Layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)

fig5_dict = {
    'Year': year,
    'Key': labels,
    'Budget / Expense Value': values
}

df5 = pd.DataFrame(fig5_dict)
fig5 = px.bar(df5, x='Year', y='Budget / Expense Value', color='Key', title='BPDs Adopted Budget, Appropriated Budget, and Actual Expenses', barmode="group", color_discrete_sequence=['#ffe06c','#6c8bff','#ff6c8b'])
fig5.update_layout(plot_bgcolor='rgba(0,0,0,0)')
fig5.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='LightGray')
fig5.update_traces(marker_line_width = 0)
fig5.show()